In [ ]:
import tensorflow as tf

In [ ]:
# three triangular membership functions
def membership_small(x, c=1.0, d=3.0):
  return tf.maximum(0.0, tf.minimum(1.0, (d-x)/(d-c)))

def membership_medium(x, a=1.5, m=3.0, b=4.5):
  return tf.maximum(0.0, tf.minimum((x-a)/(m-a), (b-x)/(b-m)))

def membership_large (x, r=3.0, q=4.5):
  return tf.maximum(0.0, tf.minimum(1.0, (x-r)/(q-r)))

In [ ]:
# fuzzy pooling operation
def fuzzy_pooling(input_tensor, ksize=2, stride=2):
    # extract patches from input tensor
    patches = tf.image.extract_patches(
        images=input_tensor,
        sizes=[1, ksize, ksize, 1],
        strides=[1, stride, stride, 1],
        rates=[1, 1, 1, 1],
        padding='VALID'
    )
    batch_size = tf.shape(input_tensor)[0]
    height = tf.shape(patches)[1]
    width = tf.shape(patches)[2]
    channels = tf.shape(input_tensor)[3]
    patches = tf.reshape(patches, (batch_size, height, width, ksize, ksize, channels))

    # apply fuzzy sets
    small = membership_small(patches)
    medium = membership_medium(patches)
    large = membership_large(patches)

    # fuzzy aggregation
    small_score = tf.reduce_sum(small, axis=[3,4])
    medium_score = tf.reduce_sum(medium, axis=[3,4])
    large_score = tf.reduce_sum(large, axis=[3,4])

    # defuzzification (using center of gravity)
    defuzz_value = (tf.reduce_sum(small * patches, axis=[3, 4]) +
                  tf.reduce_sum(medium * patches, axis=[3, 4]) +
                  tf.reduce_sum(large * patches, axis=[3, 4])) / (small_score + medium_score + large_score + 1e-7)

    return defuzz_value

In [ ]:
# fuzzy pooling layer
class FuzzyPoolingLayer(tf.keras.layers.Layer):
    def __init__(self, pool_size=(2,2), strides=(2,2)):
      super(FuzzyPoolingLayer, self).__init__()
      self.pool_size = pool_size
      self.strides = strides

    def call(self, inputs):
      return fuzzy_pooling(inputs, ksize=self.pool_size[0], stride=self.strides[0])

In [ ]:
# CNN model with fuzzy pooling layer
def build_fuzzy_pooling_cnn(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        FuzzyPoolingLayer(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    return model

In [ ]:
# load MNIST dataset
(train_images_mnist, train_labels_mnist), (test_images_mnist, test_labels_mnist) = tf.keras.datasets.mnist.load_data()

# preprocess data
train_images_mnist = train_images_mnist.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images_mnist = test_images_mnist.reshape((10000, 28, 28, 1)).astype('float32') / 255

# build the model
model_fuzzy = build_fuzzy_pooling_cnn((28, 28, 1))

# compile the model
model_fuzzy.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# load MNIST dataset
(train_images_mnist, train_labels_mnist), (test_images_mnist, test_labels_mnist) = tf.keras.datasets.mnist.load_data()

# preprocess data
train_images_mnist = train_images_mnist.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images_mnist = test_images_mnist.reshape((10000, 28, 28, 1)).astype('float32') / 255

# build the model
model_fuzzy = build_fuzzy_pooling_cnn((28, 28, 1))

# compile the model
model_fuzzy.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train the model
model_fuzzy.fit(train_images_mnist, train_labels_mnist, epochs=5, batch_size=32)

# evaluate on test data
test_loss_fuzzy_mnist, test_acc_fuzzy_mnist = model_fuzzy.evaluate(test_images_mnist, test_labels_mnist)
print('Test accuracy of fuzzy pooling on MNIST dataset:', test_acc_fuzzy_mnist)

In [ ]:
# CNN model with max pooling layer
def build_max_pooling_2D_cnn(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

# build the model
model_max = build_max_pooling_2D_cnn((28, 28, 1))

# compile the model
model_max.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train the model
model_max.fit(train_images_mnist, train_labels_mnist, epochs=5, batch_size=32)

# evaluate on test data
test_loss_max_mnist, test_acc_max_mnist = model_max.evaluate(test_images_mnist, test_labels_mnist)
print('Test accuracy of max pooling on MNIST dataset:', test_acc_max_mnist)



In [ ]:
# CNN model with average pooling layer
def build_avg_pooling_cnn(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.AveragePooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

# build the model
model_avg = build_avg_pooling_cnn((28, 28, 1))

# compile the model
model_avg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train the model
model_avg.fit(train_images_mnist, train_labels_mnist, epochs=5, batch_size=32)

# evaluate on test data
test_loss_avg_mnist, test_acc_avg_mnist = model_avg.evaluate(test_images_mnist, test_labels_mnist)
print('Test accuracy of average pooling on MNIST dataset:', test_acc_avg_mnist)


In [ ]:
# load Fashion-MNIST
(train_images_fmnist, train_labels_fmnist), (test_images_fmnist, test_labels_fmnist) = tf.keras.datasets.fashion_mnist.load_data()

# preprocess data
train_images_fmnist = train_images_fmnist.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images_fmnist = test_images_fmnist.reshape((10000, 28, 28, 1)).astype('float32') / 255

# train the model
model_fuzzy.fit(train_images_fmnist, train_labels_fmnist, epochs=5, batch_size=32)

# evaluate on test data
test_loss_fuzzy_fmnist, test_acc_fuzzy_fmnist = model_fuzzy.evaluate(test_images_fmnist, test_labels_fmnist)
print('Test accuracy of fuzzy pooling on Fashion-MNIST dataset:', test_acc_fuzzy_fmnist)

In [ ]:
# train the model
model_max.fit(train_images_fmnist, train_labels_fmnist, epochs=5, batch_size=32)

# evaluate on test data
test_loss_max_fmnist, test_acc_max_fmnist = model_max.evaluate(test_images_fmnist, test_labels_fmnist)
print('Test accuracy of max pooling on Fashion-MNIST dataset:', test_acc_max_fmnist)

In [ ]:
# train the model
model_avg.fit(train_images_fmnist, train_labels_fmnist, epochs=5, batch_size=32)

# evaluate on test data
test_loss_avg_fmnist, test_acc_avg_fmnist = model_avg.evaluate(test_images_fmnist, test_labels_fmnist)
print('Test accuracy of average pooling on Fashion-Mnist dataset:', test_acc_avg_fmnist)

In [ ]:
# CNN model with fuzzy pooling layer for CIFAR-10
def build_fuzzy_pooling_cnn(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        FuzzyPoolingLayer(pool_size=(2, 2)),  # first fuzzy pooling layer
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        FuzzyPoolingLayer(pool_size=(2, 2)),  # second fuzzy pooling layer
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

# load CIFAR-10
(train_images_cifar, train_labels_cifar), (test_images_cifar, test_labels_cifar) = tf.keras.datasets.cifar10.load_data()

# preprocess data
train_images_cifar = train_images_cifar.astype('float32') / 255.0
test_images_cifar = test_images_cifar.astype('float32') / 255.0

# build and compile the model
model_fuzzy_cifar = build_fuzzy_pooling_cnn((32, 32, 3))
model_fuzzy_cifar.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train the model
model_fuzzy_cifar.fit(train_images_cifar, train_labels_cifar, epochs=5, batch_size=32)

# evaluate on test data
test_loss_fuzzy_cifar, test_acc_fuzzy_cifar = model_fuzzy_cifar.evaluate(test_images_cifar, test_labels_cifar)
print('Test accuracy of fuzzy pooling on CIFAR-10 dataset:', test_acc_fuzzy_cifar)


In [ ]:
# CNN model with max pooling layer for CIFAR-10
def build_avg_pooling_cnn(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

# build and compile the model
model_max_cifar = build_avg_pooling_cnn((32, 32, 3))
model_max_cifar.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train the model
model_max_cifar.fit(train_images_cifar, train_labels_cifar, epochs=5, batch_size=32)

# evaluate on test data
test_loss_max_cifar, test_acc_max_cifar = model_max_cifar.evaluate(test_images_cifar, test_labels_cifar)
print('Test accuracy of max pooling on CIFAR-10 dataset:', test_acc_max_cifar)


In [ ]:
# CNN model with average pooling layer for CIFAR-10
def build_avg_pooling_cnn(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.AveragePooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

# build and compile the model
model_avg_cifar = build_avg_pooling_cnn((32, 32, 3))
model_avg_cifar.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train the model
model_avg_cifar.fit(train_images_cifar, train_labels_cifar, epochs=5, batch_size=32)

# evaluate on test data
test_loss_avg_cifar, test_acc_avg_cifar = model_avg_cifar.evaluate(test_images_cifar, test_labels_cifar)
print('Test accuracy of average pooling on CIFAR-10 dataset:', test_acc_avg_cifar)
